# INMARSAT April Bill 2021

## Import Libraries

In [1]:
import pandas as pd

import numpy as np

import re

import os

## Cleaning Data: 
1. Uploading Invoice, 2. Removing NaNs, 3. Renaming Columns Needed, 4. Filtering Totals, 5. Resetting Index To Current Filtered Rows, 6. Removing 'Unamed' Columns, 7. Getting ID Numbers For New 'IDs' Column, 8. Getting Amounts Creating New 'Amounts Due' Column, 9. Turning Amounts Due from Strings of Text into Numbers, 10. Removing 'ID' and 'Totals' Column, 11. Saving Version to .csv, 12. Checking My Work, 13. Checking Amounts Due Totals

In [2]:
df = pd.read_csv('April_2021_Invoice.csv') #1. Uploading Invoice

df = df.fillna("") #2. Removing NaNs

df = df.rename(columns={"Unnamed: 8": "ID", "Unnamed: 20": "Totals"}) #3. Renaming Columns Needed
df = df.loc[df['ID'].str.contains('^to[a-z]*', flags=re.I, regex=True)] #4. Filtering Totals
df = df.reset_index(drop=True) #5. Resetting Index To Current Filtered Rows

df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #6. Removing 'Unamed' Columns
def get_ID(ID):
    return ID.split(' ')[2] #7. Getting ID Numbers For New 'IDs' Column

df['IDs'] = df['ID'].apply(lambda x: get_ID(x)) #7. Getting ID Numbers For New 'IDs' Column

df['Amounts Due'] = df['Totals'].apply(lambda x: x.split('$')[1]) #8. Getting Amounts Creating New 'Amounts Due' Column

df['Amounts Due'] = pd.to_numeric(df['Amounts Due']) #9. Turning Amounts Due from Strings of Text into Numbers

df = df.drop(columns=['ID', 'Totals']) #10. Removing 'ID' and 'Totals' Column

df.to_excel('INMARSAT April 2021.xlsx', index=False) #11. Saving Version to CSV

df

,Bill To:,IDs,Amounts Due
0,10/11/2019 -,000000000100,13.79
1,,000000000101,13.79
2,,000000000102,13.79
3,,000000000103,13.79
4,,000000000104,13.79
...,...,...,...
520,,881651461225,15.44
521,,881651477444,15.44
522,,881651477445,39.19
523,,881651477579,19.44


In [3]:
df.sum(axis = 0, skipna = True) #13. Checking Amounts Due Totals

Bill To:       10/11/2019 -10/11/2019 -10/11/2019 -10/11/2019...
IDs            0000000001000000000001010000000001020000000001...
Amounts Due                                              13831.7
dtype: object

## Merging .csv Files Based on ID Numbers
1. Defining Our Previous Notebook Saved, 2. PI and ID Number Data, 3. Uploading Data, 4. Printing to see our columns, 5. Merging Our Files Based on ID Number, 6. Saving Version to .csv, 7. Checking My Work, 8. Checking Amounts Due Totals

In [4]:
initial_workbook = 'INMARSAT April 2021.xlsx' #1. Defining Our Previous Notebook Saved
info_workbook = 'Active DF COMPLETE 26 May.xlsx' #2. Defining PI and ID Number Dataframe

df_initial = pd.read_excel(initial_workbook) #3. Uploading Data
df_info = pd.read_excel(info_workbook) #3. Uploading Data

print(df_initial.columns) #4. Printing to see our columns
print(df_info.columns) #4. Printing to see our columns


Index(['Bill To:', 'IDs', 'Amounts Due'], dtype='object')
Index(['IDs', 'ICCID', 'MSISDN-C', 'Type', 'PI'], dtype='object')


In [5]:
df_3 = pd.merge(df_initial, df_info[['IDs','ICCID','MSISDN-C','PI','Type']], on='IDs', how='left') #5. Merging Our Files Based on ID Number
df_3.to_excel('INMARSAT April 2021 Data.xlsx', index=False) #6. Saving Version to .csv

df_3 #7. Checking My Work


,Bill To:,IDs,Amounts Due,ICCID,MSISDN-C,PI,Type
0,10/11/2019 -,100,13.79,3.000340e+14,3.000340e+14,Sutton,SHORT BURST DATA
1,NaN,101,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
2,NaN,102,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
3,NaN,103,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
4,NaN,104,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
...,...,...,...,...,...,...,...
520,NaN,881651461225,15.44,8.988170e+18,8.816938e+11,McPhaden,RUDICS
521,NaN,881651477444,15.44,8.988170e+18,8.816938e+11,Sutton,RUDICS
522,NaN,881651477445,39.19,8.988170e+18,8.816938e+11,Sutton,RUDICS
523,NaN,881651477579,19.44,8.988170e+18,8.816938e+11,Sutton,RUDICS


In [6]:
df_3.sum(axis = 0, skipna = True) #8. Checking Amounts Due Totals

IDs                                              386154177801126
Amounts Due                                              13831.7
ICCID                                                3.88292e+21
MSISDN-C                                             2.64606e+16
PI             SuttonMcPhadenMcPhadenMcPhadenMcPhadenMcPhaden...
Type           SHORT BURST DATASHORT BURST DATASHORT BURST DA...
dtype: object

## Generating PI and Type Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI' and 'Type', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [7]:
df_3 = pd.read_excel('INMARSAT April 2021 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "Apr'21 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI', 'Type']).sum() #3. Organizing Based on 'PI' and 'Type'

df_3 = df_3.drop(columns=['IDs','ICCID', 'MSISDN-C']) #4. Removing Column

df_3.to_excel('INMARSAT April 2021 BILL PI TYPES.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

Apr'21 Amounts Due
PI                                       Type                                
Cronin                                   RUDICS                        334.16
Cross                                    RUDICS                        173.71
                                         SHORT BURST DATA               18.59
Dziak                                    Non-RUDICS                     47.99
                                         RUDICS                         15.44
Feely                                    RUDICS                         93.20
                                         SHORT BURST DATA               13.79
Johnson                                  RUDICS                       6999.08
                                         SHORT BURST DATA              237.51
Klinck                                   RUDICS                         28.69
McPhaden                                 RUDICS                       1791.78
                                         SHORT BURST DATA              275.80
Meinig - Dziak                           RUDICS                         30.88
Meinig - Dziak DEACTIVATED 4/13/2021     RUDICS                         15.44
Meinig - Feely                           RUDICS                         30.88
Meinig Chaba                             RUDICS                         25.44
Meinig FCOM1.0                           RUDICS                         15.44
Meinig Former DART                       RUDICS                        227.26
                                         SHORT BURST DATA               27.58
Meinig Former DART DEACTIVATED 4/20/2021 RUDICS VOICE                   47.99
                                         SHORT BURST DATA               55.16
Meinig GNSS GPS                          SHORT BURST DATA               13.79
Meinig SPURS2                            RUDICS                         15.44
Meinig SPURS2 DEACTIVATED 04/20/2021     RUDICS                         23.44
                                         SHORT BURST DATA               27.58
Meinig pop_up                            RUDICS                        217.22
Mordy IOTAS-ITAE DEACTIVATED 4/20/2021   SHORT BURST DATA              179.27
Mordy ITAE                               RUDICS                        170.95
Stabeno                                  RUDICS                         20.19
                                         SHORT BURST DATA              387.19
Sutton                                   Non-RUDICS                     15.44
                                         RUDICS                       2105.57
                                         SHORT BURST DATA              103.37
Sutton DEACTIVATED 5/06/2021             Non-RUDICS                     46.44

In [8]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

Apr'21 Amounts Due    13831.7
dtype: float64

## Generating PI Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [9]:
df_3 = pd.read_excel('INMARSAT April 2021 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "Apr'21 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI']).sum() #3. Organizing Based on 'PI'

df_3 = df_3.drop(columns=['IDs','ICCID', 'MSISDN-C']) #4. Removing Column

df_3.to_excel('INMARSAT April 2021 BILL PI.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

,Apr'21 Amounts Due
PI,
Cronin,334.16
Cross,192.30
Dziak,63.43
Feely,106.99
Johnson,7236.59
Klinck,28.69
McPhaden,2067.58
Meinig - Dziak,30.88
Meinig - Dziak DEACTIVATED 4/13/2021,15.44


In [10]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

Apr'21 Amounts Due    13831.7
dtype: float64